In [1]:
import sys
sys.path.insert(1, 'C:/Users/raena/Documents/Imperial College London/msc_thesis/code/functions_and_classes')

from dataRead import *
from glob import glob
import pandas as pd
import numpy as np

## Index Period 4

Backtest:
* Jan 2017 - Dec 2022
* Major event: Covid-19 pandemic

In [2]:
# Get DJI Index weights data per quarter

DJI_weights_paths = glob('../../data/dataBBG/DJI_index_weights/DJI_*.pickle')
DJI_weights_per_quarter = {}
for path in DJI_weights_paths:
    date = path.split('.')[-2].split('_')[-1]
    DJI_weights_per_quarter[date] = pd.read_pickle(path)

# Import price and volume data for each stock in the DJI Index
DJI_PX_LAST_paths = glob('../../data/dataBBG/DJI_stock_PX_LAST/* Equity.pickle')
DJI_PX_VOLUME_paths = glob('../../data/dataBBG/DJI_stock_PX_VOLUME/* Equity.pickle')

DJI_price_data = {}
DJI_volume_data = {}

for path in DJI_PX_LAST_paths:
    ticker = (' ').join(path.split('.')[-2].split('\\')[-1].split(' ')[0:1])
    # if ticker == '3277Q UN': # JP Morgan
    #     path = '../../data/dataBBG/DJI_stock_PX_LAST/JPM UN Equity.pickle'
    df = pd.read_pickle(path).set_index('date')
    df.index = pd.to_datetime(df.index)
    DJI_price_data[ticker] = df

for path in DJI_PX_VOLUME_paths:
    ticker = (' ').join(path.split('.')[-2].split('\\')[-1].split(' ')[0:1])
    df = pd.read_pickle(path).set_index('date')
    df.index = pd.to_datetime(df.index)
    # df.index = pd.to_datetime(df.index)
    DJI_volume_data[ticker] = df

trading_days = pd.to_datetime(list(DJI_price_data['AAPL'].index))

In [3]:
start_day = '2017-01-01'
end_day = '2022-12-31'
start_day_count = np.where(trading_days >= start_day)[0][0]
end_day_count = np.where(trading_days <= end_day)[0][-1]

In [4]:
windowOptions = [20,30,60]

validation_start_day = '2022-01-01'
validation_end_day = end_day
validation_start_day_count = np.where(trading_days >= validation_start_day)[0][0]
validation_end_day_count = np.where(trading_days <= validation_end_day)[0][-1]

In [5]:
performance_metrics = ['Annual. Returns',
                    'Annual. Stand. Dev',
                    'Annual. Skew',
                    'Annual. Kurtosis',
                    'Total Returns',
                    'Arith. Returns',
                    'Geom. Returns',
                    'Sharpe Ratio',
                    'Max. Drawdown',
                    'Annual. Turnover',
                    'VaR']

In [6]:
from priceVolumeIndexStrategyMinVariance import *

validationResultsDict = {}
performanceResults = {}

for windowSize in windowOptions:
    # 20, 30, 60
    start = validation_start_day_count - (windowSize + 1)
    end = validation_end_day_count
    trading_days_subset = trading_days[start:end]
    
    sInst = PriceVolumeIndexMinVarStrategy(
                indexDict = DJI_weights_per_quarter,
                trading_days = trading_days_subset,
                priceDict = DJI_price_data,    
                volumeDict = DJI_volume_data,
                lookbackWindow = windowSize,
                numberOfMembers = 30,
                factor = 252.0)
    sInst = sInst.getStrategyWeights(
        methods = ['HIS','LW', 'IDN'])

    b = sInst.backtestStrategy()
    
    methodList = list(b[windowSize+1].keys())
    methodDict = {'Portfolio Returns': {}, 'Portfolio Value': {}, 'Volume Bought': {}, 'Volume Sold' : {}}
    for method in methodList:
        methodDict['Portfolio Returns'][method] = pd.DataFrame([b[k][method] for k in b.keys()])['Portfolio Returns']
        methodDict['Portfolio Value'][method] = pd.DataFrame([b[k][method] for k in b.keys()])['Portfolio Value']
        methodDict['Volume Bought'][method] = pd.DataFrame([b[k][method] for k in b.keys()])['Volume Bought']
        methodDict['Volume Sold'][method] = pd.DataFrame([b[k][method] for k in b.keys()])['Volume Sold']

    # validationResultsDict[key] = methodDict

    for method in methodList:
        meth = method
        retS = methodDict['Portfolio Returns'][meth]
        dateIndex = trading_days_subset[-retS.shape[0]:]
        retS.index = dateIndex
        valS = methodDict['Portfolio Value'][meth]
        valS.index = dateIndex
        volBS = methodDict['Volume Bought'][meth]
        volBS.index = dateIndex
        volSS = methodDict['Volume Sold'][meth]
        volSS.index = dateIndex

        from performanceAnalysis import *
        perfInst = PerformanceAnalysis(portfolioReturns = retS,
                                    portfolioValue = valS,
                                    volBought = volBS,
                                    volSold = volSS,
                                    factor = 252.0)
        perfRes = perfInst.metricSeries(rf = 0)
        key = method + '_' + str(windowSize)
        performanceResults[key] = {met : {} for met in performance_metrics}
        for metric in performance_metrics:
            performanceResults[key][metric] = perfRes.loc[metric]

    print('\n')

C:\Users\raena\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 249/249 [00:04<00:00, 50.27it/s]


100%|██████████| 249/249 [00:04<00:00, 51.56it/s]


100%|██████████| 249/249 [00:04<00:00, 53.34it/s]

In [7]:
pRes1 = pd.DataFrame(performanceResults).T[['Annual. Stand. Dev', 'Max. Drawdown', 'VaR']]
pRes1 = pRes1.style.format(precision = 4)
print('\n'.join(str(pRes1.to_latex()).split('\n')))

\begin{tabular}{lrrr}
 & Annual. Stand. Dev & Max. Drawdown & VaR \\
HIS_20 & 0.1686 & -0.2013 & -0.0168 \\
LW_20 & 0.1554 & -0.1847 & -0.0163 \\
IDN_20 & 0.2005 & -0.2289 & -0.0223 \\
HIS_30 & 0.1586 & -0.1694 & -0.0161 \\
LW_30 & 0.1537 & -0.1663 & -0.0158 \\
IDN_30 & 0.2005 & -0.2289 & -0.0223 \\
HIS_60 & 0.1542 & -0.2093 & -0.0163 \\
LW_60 & 0.1522 & -0.2118 & -0.0155 \\
IDN_60 & 0.2005 & -0.2289 & -0.0223 \\
\end{tabular}



In [8]:
# fp = '../../code/min_variance/'
# fname = "period_4_" + str(windowOptions[0])
# pd.DataFrame(performanceResults).to_pickle(f'{fp}/{fname}.pickle')

In [9]:
from pyfinance import TSeries

a = pd.Series(TSeries((DJI_price_data['AAPL'])['PX_LAST']).rollup('M'))
